In [295]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math, decimal
import requests
dec = decimal.Decimal
import hvplot.pandas
import krakenex
from pykrakenapi import KrakenAPI
api = krakenex.API()
k = KrakenAPI(api)
import json
import yfinance as yf
FearGreedUrl = requests.get('https://api.alternative.me/fng/?limit=2').json()

In [296]:
from pathlib import Path
csvpath1 = Path("./full_moon.csv")
lunar_eclipse = Path("./lunar_eclipse.csv")
solar_eclipse = Path("./solar_eclipse.csv")

# Retrieve BTC Price from Kraken API

In [297]:
ohlc = k.get_ohlc_data('BTCUSD', interval=1440, ascending = True)
Price = ohlc[0]['Price'] = ohlc[0]['close']
SMA_8 = ohlc[0]['8SMA'] = ohlc[0]['close'].rolling(8).mean()
SMA_200 = ohlc[0]['200SMA'] = ohlc[0]['close'].rolling(200).mean()
EMA_200 = ohlc[0]['200EMA'] = ohlc[0]['close'].ewm(200).mean()
ta_df = ohlc[0].drop(['high', 'low', 'time', 'open', 'close', 'vwap', 'count', 'volume'], axis = 1)
# ta_df.plot(figsize = (25,10))
ta_df.head()

,Price,8SMA,200SMA,200EMA
dtime,,,,
2020-07-03,9058.0,NaN,NaN,9058.000000
2020-07-04,9138.5,NaN,NaN,9098.350374
2020-07-05,9083.8,NaN,NaN,9093.476039
2020-07-06,9348.2,NaN,NaN,9157.634236
2020-07-07,9256.8,NaN,NaN,9177.665718


### Plot BTC Price over Time

In [298]:
BTC = ohlc[0].hvplot.line(
    x = 'dtime',
    y = 'Price',
    xlabel = 'Date',
    ylabel = 'Price',
    title = 'BTC Price',
    legend = True
)
BTC

:Curve   [dtime]   (Price)

# Clean Lunar Data


In [299]:
#Create a Full Moon Dataframe
full_moon = pd.read_csv(csvpath1, parse_dates=True, index_col=' Date', infer_datetime_format=True)
full_moon.reset_index(inplace=True)
full_moon = full_moon.rename(columns = {' Date':'Full_Moon'})
full_moon.drop(['Day',' Time'], axis=1, inplace=True)

In [300]:
#Create a New Moon column
full_moon['New_Moon'] = full_moon['Full_Moon'] + timedelta(days=15)

In [301]:
lunar_eclipse_df = pd.read_csv(lunar_eclipse, parse_dates=True, infer_datetime_format=True)
lunar_eclipse_df.rename(columns={'Date':'lunar eclipse'}, inplace=True)
lunar_eclipse_df['lunar eclipse'] = pd.to_datetime(lunar_eclipse_df['lunar eclipse'])

lunar_eclipse_df = lunar_eclipse_df.join(ohlc[0]['Price'], on = 'lunar eclipse', how = 'inner')

In [302]:
solar_eclipse_df = pd.read_csv(solar_eclipse, parse_dates=True, infer_datetime_format=True)
solar_eclipse_df.rename(columns={'Date':'solar eclipse'}, inplace=True)
solar_eclipse_df['solar eclipse'] = pd.to_datetime(solar_eclipse_df['solar eclipse'])
solar_eclipse_df.drop(['Unnamed: 1'], axis = 1, inplace=True)

solar_eclipse_df = solar_eclipse_df.join(ohlc[0]['Price'], on = 'solar eclipse', how = 'inner')

In [303]:
# Create a Moon Data table with the BTC Price data for each Full and New Moon date

full_moon = full_moon.join(ohlc[0]['Price'], on = 'Full_Moon', how = 'inner')
full_moon['Full Moon Price'] = full_moon['Price']
full_moon = full_moon.drop(columns=['Price']).reset_index()

full_moon = full_moon.join(ohlc[0]['Price'], on = 'New_Moon', how = 'inner')
full_moon['New Moon Price'] = full_moon['Price']
full_moon = full_moon.drop(columns=['Price']).reset_index()

moon_data = full_moon.drop(columns=['index','level_0'])

moon_data.head()

,Full_Moon,New_Moon,Full Moon Price,New Moon Price
0,2020-07-05,2020-07-20,9083.8,9163.0
1,2020-08-03,2020-08-18,11237.3,11957.0
2,2020-09-02,2020-09-17,11396.9,10944.8
3,2020-10-01,2020-10-16,10616.1,11326.9
4,2020-10-31,2020-11-15,13809.1,15962.3


# Plot Lunar Data over BTC Data

In [304]:
# create hvplot figures to then overaly 
glyph_1 = ohlc[0]['Price'].hvplot.line( 
    'dtime', 'Price',
    color='#e7e7e7',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_2 = moon_data.hvplot.scatter(
    x = 'Full_Moon',
    y = 'Full Moon Price',
    color='#ffcd33',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_3 = moon_data.hvplot.scatter(
    x = 'New_Moon',
    y = 'New Moon Price',
    color='#ff6533',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_4 = ohlc[0]['8SMA'].hvplot.line(
    'dtime', '8SMA',
    color='#70eac4',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_5 = ohlc[0]['200SMA'].hvplot.line(
    'dtime', '200SMA',
    color='#55d24a',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_6 = ohlc[0]['200EMA'].hvplot.line(
    'dtime', '200EMA',
    color='#22a91a',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_7 = lunar_eclipse_df.hvplot.scatter(
    x = 'lunar eclipse',
    y = 'Price',
    color='black',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_8 = solar_eclipse_df.hvplot.scatter(
    x = 'solar eclipse',
    y = 'Price',
    color='black',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_1*glyph_2*glyph_3*glyph_4*glyph_5*glyph_6*glyph_7*glyph_8

:Overlay
   .Curve.Price    :Curve   [dtime]   (Price)
   .Scatter.I      :Scatter   [Full_Moon]   (Full Moon Price)
   .Scatter.II     :Scatter   [New_Moon]   (New Moon Price)
   .Curve.A_8SMA   :Curve   [dtime]   (8SMA)
   .Curve.A_200SMA :Curve   [dtime]   (200SMA)
   .Curve.A_200EMA :Curve   [dtime]   (200EMA)
   .Scatter.III    :Scatter   [lunar eclipse]   (Price)
   .Scatter.IV     :Scatter   [solar eclipse]   (Price)

## Prepare Lunar and BTC Data for Merge

In [305]:
ohlc = k.get_ohlc_data('BTCUSD', interval=1440, ascending = True)
Price = ohlc[0]['Price'] = ohlc[0]['close']
SMA_8 = ohlc[0]['8SMA'] = ohlc[0]['close'].rolling(8).mean()
SMA_200 = ohlc[0]['200SMA'] = ohlc[0]['close'].rolling(200).mean()
EMA_200 = ohlc[0]['200EMA'] = ohlc[0]['close'].ewm(200).mean()
ohlc[0].drop(['high', 'low', 'time', 'open', 'close', 'vwap', 'count', 'volume'], axis = 1, inplace=True)
ohlc = ohlc[0]
ohlc.reset_index(inplace=True)
ohlc['dtime'] = pd.to_datetime(ohlc['dtime'])
ohlc.tail()

,dtime,Price,8SMA,200SMA,200EMA
715,2022-06-18,18950.0,22728.7125,39763.6725,40480.184574
716,2022-06-19,20555.5,21750.6250,39580.2605,40378.202775
717,2022-06-20,20550.0,20998.1750,39400.3785,40276.729339
718,2022-06-21,20701.9,20778.4000,39235.4950,40176.566849
719,2022-06-22,20381.4,20562.2000,39091.1265,40075.291272


In [306]:
# Read Full_Moon CSV, Clean index, infer Datetime
full_moon = pd.read_csv(csvpath1, parse_dates=True, index_col=' Date', infer_datetime_format=True)
full_moon.reset_index(inplace=True)
full_moon = full_moon.rename(columns = {' Date':'dtime'})
full_moon.drop(['Day',' Time'], axis=1, inplace=True)
full_moon['Phase'] = full_moon.loc['dtime',:] = 'Full Moon'
full_moon.drop(full_moon.index[-1], inplace=True)

In [307]:
# Timedelta + 14 days to create New_Moon Dataframe
new_moon = pd.DataFrame(full_moon['dtime'] + timedelta(days=15))
new_moon['Phase'] = new_moon.loc['dtime',:] = 'New Moon'
new_moon.drop(new_moon.index[-1], inplace=True)

In [308]:
# Append Lunar Dataframes to create one table sorted by 'dtime'
phase_data = full_moon.append(new_moon)
phase_data.sort_values('dtime', inplace= True)
phase_data.reset_index(inplace=True)
phase_data.drop(['index'], axis = 1, inplace=True)
phase_data['dtime'] = pd.to_datetime(phase_data['dtime'])
phase_data.columns
phase_data

,dtime,Phase
0,1900-01-15,Full Moon
1,1900-01-30,New Moon
2,1900-02-14,Full Moon
3,1900-03-01,New Moon
4,1900-03-16,Full Moon
...,...,...
3731,2050-11-14,New Moon
3732,2050-11-28,Full Moon
3733,2050-12-13,New Moon
3734,2050-12-28,Full Moon


In [309]:
eclipse_df1 = pd.read_csv(lunar_eclipse, parse_dates=True, infer_datetime_format=True)
eclipse_df1.rename(columns={'Date':'dtime'}, inplace=True)
eclipse_df1['lunar eclipse'] = 1
eclipse_df1['dtime'] = pd.to_datetime(eclipse_df1['dtime'])
eclipse_df1.set_index(['dtime'])

,lunar eclipse
dtime,
1998-08-08,1
1999-01-31,1
1999-07-28,1
2000-01-21,1
2000-07-16,1
...,...
2098-10-10,1
2099-04-05,1
2099-09-29,1


In [310]:
eclipse_df2 = pd.read_csv(solar_eclipse, parse_dates=True, infer_datetime_format=True)
eclipse_df2.rename(columns={'Date':'dtime'}, inplace=True)
eclipse_df2['solar eclipse'] = 1
eclipse_df2['dtime'] = pd.to_datetime(eclipse_df1['dtime'])
eclipse_df2.drop(['Unnamed: 1'], axis = 1, inplace=True)
eclipse_df2.set_index(['dtime'])

,solar eclipse
dtime,
1998-08-08,1
1999-01-31,1
1999-07-28,1
2000-01-21,1
2000-07-16,1
...,...
2094-01-01,1
2094-06-28,1
2094-12-21,1


# Merge Dataframes on 'dtime'

In [311]:
moon_merge = phase_data.merge(ohlc, on='dtime', how='inner')
moon_merge.set_index(['dtime'], inplace=True)
# moon_merge.tail()

In [312]:
eclipse_cat = moon_merge.merge(eclipse_df1, on='dtime', how='outer')
eclipse_cat['lunar eclipse'] = eclipse_cat['lunar eclipse'].fillna(0)
eclipse_cat.set_index(['dtime'], inplace=True)
eclipse_cat.dropna()

,Phase,Price,8SMA,200SMA,200EMA,lunar eclipse
dtime,,,,,,
2021-01-28,Full Moon,33430.4,32106.8875,16967.3415,18878.305151,0.0
2021-02-12,New Moon,47470.4,43704.7375,19230.8385,21111.635282,0.0
2021-02-27,Full Moon,46170.2,50718.1875,22164.4225,24152.296531,0.0
2021-03-14,New Moon,59001.1,56167.1500,25228.6335,27011.233582,0.0
2021-03-28,Full Moon,55782.0,54516.2625,28371.4535,29639.153652,0.0
2021-04-12,New Moon,59823.7,58595.9750,31957.8260,32383.281165,0.0
2021-04-27,Full Moon,55078.7,52694.3000,35377.0710,34602.003118,0.0
2021-05-12,New Moon,49502.7,56335.8000,38694.7220,36562.406453,0.0
2021-05-26,Full Moon,39283.9,37922.6875,40630.7140,36979.060696,1.0


In [313]:
def FM_result(x, y):
    if x - y < 0:
        return True
    return False

moon_data['FM Result'] = moon_data.apply(lambda row: FM_result(row['Full Moon Price'], row['New Moon Price']), axis=1)
moon_data['FM Percentage Difference'] = (moon_data['Full Moon Price'] - moon_data['New Moon Price']) / moon_data['Full Moon Price']

def NM_result(x, y):
    if x - y > 0:
        return True
    return False

moon_data['NM Result'] = moon_data.apply(lambda row: NM_result(row['New Moon Price'], row['Full Moon Price']), axis=1)
moon_data['NM Percentage Difference'] = (moon_data['New Moon Price'] - moon_data['Full Moon Price']) / moon_data['New Moon Price']

#for ind in (moon_data.index):
#print(moon_data.iloc[:,2])
#print(moon_data.iloc[1:,3]- moon_data.iloc[:,2])


moon_data.head()

,Full_Moon,New_Moon,Full Moon Price,New Moon Price,FM Result,FM Percentage Difference,NM Result,NM Percentage Difference
0,2020-07-05,2020-07-20,9083.8,9163.0,True,-0.008719,True,0.008643
1,2020-08-03,2020-08-18,11237.3,11957.0,True,-0.064046,True,0.060191
2,2020-09-02,2020-09-17,11396.9,10944.8,False,0.039669,False,-0.041307
3,2020-10-01,2020-10-16,10616.1,11326.9,True,-0.066955,True,0.062753
4,2020-10-31,2020-11-15,13809.1,15962.3,True,-0.155926,True,0.134893


In [314]:
# Create a New Temporary Data Frame to Finalize the Moon Price Behavior Data

frame = {'Full Moon': moon_data.iloc[:,0], 'New Moon': moon_data.iloc[:,1], 'Full Moon Price': moon_data.iloc[:,2], 'New Moon Price': moon_data.iloc[:,3]} #'New Moon Price': moon_data.iloc[1:,3]}
tmpdf = pd.DataFrame(frame)
tmpdf['Difference']= tmpdf['New Moon Price']-tmpdf['Full Moon Price'] #.shift(1)
tmpdf['Percentage Difference'] = ((tmpdf['Full Moon Price'] - tmpdf['New Moon Price']) / tmpdf['New Moon Price'])*100
tmpdf['Trade Profit']= tmpdf['Difference'].cumsum()
tmpdf['Buy and Hold']= tmpdf['New Moon Price']-tmpdf['Full Moon Price'][0]

def returnBoolFM(x):
    if x < 0:
        return True
    return False

def returnBoolNM(x):
    if x > 0:
        return True
    return False

tmpdf['Did We Profit this Month'] = tmpdf.apply(lambda row: returnBoolNM(row['Difference']), axis = 1)


tmpdf

,Full Moon,New Moon,Full Moon Price,New Moon Price,Difference,Percentage Difference,Trade Profit,Buy and Hold,Did We Profit this Month
0,2020-07-05,2020-07-20,9083.8,9163.0,79.2,-0.864346,79.2,79.2,True
1,2020-08-03,2020-08-18,11237.3,11957.0,719.7,-6.019068,798.9,2873.2,True
2,2020-09-02,2020-09-17,11396.9,10944.8,-452.1,4.130729,346.8,1861.0,False
3,2020-10-01,2020-10-16,10616.1,11326.9,710.8,-6.275327,1057.6,2243.1,True
4,2020-10-31,2020-11-15,13809.1,15962.3,2153.2,-13.489284,3210.8,6878.5,True
5,2020-11-30,2020-12-15,19708.1,19439.7,-268.4,1.380680,2942.4,10355.9,False
6,2020-12-30,2021-01-14,28879.9,39151.6,10271.7,-26.235709,13214.1,30067.8,True
7,2021-01-28,2021-02-12,33430.4,47470.4,14040.0,-29.576325,27254.1,38386.6,True
8,2021-02-27,2021-03-14,46170.2,59001.1,12830.9,-21.746883,40085.0,49917.3,True
9,2021-03-28,2021-04-12,55782.0,59823.7,4041.7,-6.756018,44126.7,50739.9,True


In [315]:
#plot Difference column vs the buy and hold column
buy_hold_glyph = tmpdf.hvplot.line(
    x = 'Full Moon',
    y = 'Buy and Hold',
    xlabel = 'Date',
    ylabel = 'Buy and Hold',
    title = 'Moon Phases Vs. Buy and Hold',
    color = 'red',
    legend = True
)

moon_glyph = tmpdf.hvplot.line(
    x = 'Full Moon',
    y = 'Trade Profit',
    xlabel = 'Date',
    ylabel = 'Difference',
    title = 'Moon Phases Vs. Buy and Hold',
    color = 'blue',
    legend = True
    
)

moon_glyph*buy_hold_glyph

:Overlay
   .Curve.I  :Curve   [Full Moon]   (Trade Profit)
   .Curve.II :Curve   [Full Moon]   (Buy and Hold)

In [316]:
# Define a function to count the Trues in the 'Did We Profit this Month' column
def count_tr(x):
    return x.sum()
number_of_Trues = (count_tr(tmpdf['Did We Profit this Month']))

#Count the number of rows in the Did We Profit this Month column

number_of_Rows = tmpdf['Did We Profit this Month'].count()

# The PERCENTAGE of times you would profit if you bought on Full Moon and sold on New Moon each month

print(f'{(number_of_Trues/number_of_Rows)*100:.2f}% is the percentage of times you would profit if you bought BTC on the full moon and sold it on the new moon of every month!')

54.17% is the percentage of times you would profit if you bought BTC on the full moon and sold it on the new moon of every month!


In [317]:
#7. create column for the all time high and low
# moon_data['All Time High'] = Price.max()
# moon_data


In [318]:
fear_data = json.dumps(FearGreedUrl["data"][0], indent=4)
fear_json = json.loads(fear_data)
fear_json


{'value': '11',
 'value_classification': 'Extreme Fear',
 'timestamp': '1655856000',
 'time_until_update': '73115'}

In [319]:
fear_df = pd.DataFrame(data=fear_json, columns=['value', 'value_classification', 'timestamp'])

ValueError: If using all scalar values, you must pass an index